In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('hexoskin_thorax_training_set_10_final.xlsx')
df

,exp_diff_mean,exp_dur_sd,duty_cycle_mean,RRV_RMSSD,ie_ratio_mean,insp_flow_sd,exp_amp_sd,insp_amp_sd,ie_ratio_sd,exp_dur_mean,Task_Label,Participant,Classification
0,0.127790,0.156604,0.492097,376.386712,0.971761,1.445912,5.304442,5.297158,0.074256,1.683594,12.0a,10785,0
1,0.201451,0.152211,0.486992,351.137516,0.950939,1.034691,2.676608,2.361021,0.057593,1.749512,12.0b,10785,0
2,0.199219,0.213642,0.476697,457.608476,0.913378,1.890795,3.564286,3.910462,0.067618,2.283203,12.0c,10785,0
3,0.285156,0.213173,0.477750,439.827676,0.918119,1.316481,2.535948,2.710758,0.082441,1.953683,12.0d,10785,0
4,0.159375,0.213828,0.481671,457.645154,0.936229,0.963488,3.092634,3.059318,0.115349,2.149088,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,0.203125,0.200555,0.490259,334.175635,0.967894,2.209139,7.984288,5.263487,0.108475,1.498828,85,98586,0
2900,1.195871,1.135998,0.408652,1022.081981,0.977483,12.234752,22.941076,24.284886,0.933998,2.233887,87,98586,1
2901,0.106337,0.117894,0.496388,185.358320,0.987026,2.990174,5.857152,5.322750,0.051696,1.456641,89,98586,0
2902,2.145182,1.447558,0.412197,3053.223554,0.748645,14.419616,19.147895,19.078216,0.283130,2.785156,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[ 0.1277902,  0.1566036,  0.4920974, ...,  5.2971579,  0.074256 ,
          1.6835938],
        [ 0.2014509,  0.1522114,  0.486992 , ...,  2.3610212,  0.0575926,
          1.7495117],
        [ 0.1992188,  0.2136419,  0.4766968, ...,  3.9104616,  0.0676182,
          2.2832031],
        ...,
        [ 0.1063368,  0.1178944,  0.4963884, ...,  5.3227497,  0.0516958,
          1.4566406],
        [ 2.1451823,  1.447558 ,  0.4121971, ..., 19.0782164,  0.2831299,
          2.7851562],
        [ 0.1235352,  0.1504568,  0.4805226, ...,  3.9285301,  0.049994 ,
          1.671875 ]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['exp_diff_mean',
  'exp_dur_sd',
  'duty_cycle_mean',
  'RRV_RMSSD',
  'ie_ratio_mean',
  'insp_flow_sd',
  'exp_amp_sd',
  'insp_amp_sd',
  'ie_ratio_sd',
  'exp_dur_mean'],
 'participants': array([10785, 10785, 10785, ..., 98586, 98586, 98586]),
 'task': array(['12.0a', '12.0b', '12.0c', ..., 89, 91, 93], dtype=object)}

In [4]:
#running the nested cross-validation for the Thorax RIP (respiratory inductance plethysmography) method using Random Forest
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

rfc = RandomForestClassifier(random_state=random_seed, class_weight="balanced")

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.96969697 0.89393939 0.87878788 0.87878788 0.98484848 0.95454545
 1.         0.86363636 0.93939394 0.90909091 0.90909091 0.8030303
 0.74242424 0.81818182 0.89393939 0.89393939 1.         0.83333333
 0.95454545 0.90909091 0.86363636 0.92424242 0.8030303  0.89393939
 0.84848485 0.98484848 0.95454545 0.98484848 0.6969697  0.75757576
 0.92424242 0.95454545 0.87878788 0.95454545 0.8030303  0.8030303
 0.90909091 0.90909091 1.         0.95454545 0.83333333 0.89393939
 0.66666667 0.87878788]
mean score:            0.8888
nested_precision_scores:    [0.96969697 0.89466089 0.8885851  0.90909091 0.98520085 0.95757576
 1.         0.88797654 0.94213287 0.92727273 0.92727273 0.85479682
 0.84922395 0.82255245 0.89525499 0.90756587 1.         0.83215369
 0.95959596 0.91831852 0.90082645 0.92398198 0.87223587 0.89525499
 0.89304813 0.98520085 0.95538803 0.98545455 0.76913876 0.75315657
 0.93730408 0.95959596 0.89747475 0.95959596 0.84022039 0.87223587
 0.90909091 0.91223776

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.9583
Specificity: 0.9762
Fold 2:
Sensitivity: 0.7917
Specificity: 0.9524
Fold 3:
Sensitivity: 0.9167
Specificity: 0.8571
Fold 4:
Sensitivity: 1.0000
Specificity: 0.8095
Fold 5:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 7:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 8:
Sensitivity: 0.9583
Specificity: 0.8095
Fold 9:
Sensitivity: 0.9583
Specificity: 0.9286
Fold 10:
Sensitivity: 1.0000
Specificity: 0.8571
Fold 11:
Sensitivity: 1.0000
Specificity: 0.8571
Fold 12:
Sensitivity: 0.9583
Specificity: 0.7143
Fold 13:
Sensitivity: 1.0000
Specificity: 0.5952
Fold 14:
Sensitivity: 0.7917
Specificity: 0.8333
Fold 15:
Sensitivity: 0.8750
Specificity: 0.9048
Fold 16:
Sensitivity: 0.9583
Specificity: 0.8571
Fold 17:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 18:
Sensitivity: 0.7500
Specificity: 0.8810
Fold 19:
Sensitivity: 1.0000
Specificity: 0.9286
Fold 20:
Sensitivity: 0.9583
Specificity: 0.8810
Fold 21:
Sensitivity: 1.0000


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8887741046831955, SD=0.07970298384169236, 95% CI=(0.865223810684857, 0.912324398681534), Min=0.6666666666666666, Max=1.0
test_precision: Mean=0.9082204891422531, SD=0.05947634022312075, 95% CI=(0.8906466765861037, 0.9257943016984024), Min=0.7531565656565656, Max=1.0
test_recall: Mean=0.8887741046831955, SD=0.07970298384169236, 95% CI=(0.865223810684857, 0.912324398681534), Min=0.6666666666666666, Max=1.0
test_f1_score: Mean=0.8841345833195832, SD=0.08141766854794594, 95% CI=(0.8600776416777018, 0.9081915249614645), Min=0.6663602941176471, Max=1.0
test_roc_auc: Mean=0.9594043109668108, SD=0.044540706823847906, 95% CI=(0.9462436149672876, 0.972565006966334), Min=0.78125, Max=1.0000000000000002
sensitivity_scores: Mean=0.9356060606060607, SD=0.08870959130963135, 95% CI=(0.9093945330682239, 0.9618175881438974), Min=0.5416666666666666, Max=1.0
specificity_scores: Mean=0.8620129870129869, SD=0.11816723699364323, 95% CI=(0.8270974413453487, 0.8969285326806251), Min=0.5, 

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 2 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 3 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 4 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 5 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 6 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 7 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 8 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 9 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: max_depth - Most frequent value: 6 (Count: 44)
Hyperparameter: max_features - Most frequent value: None (Count: 43)
Hyperparameter: max_leaf_nodes - Most frequent value: 9 (Count: 44)
Hyperparameter: n_estimators - Most frequent value: 25 (Count: 13)
